In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import sqlite3
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

print("Libraries loaded successfully!")

Libraries loaded successfully!


In [2]:
db_path = "./labeled.sqlite"

In [3]:
query = """
WITH start_end_indices AS (
    SELECT 
        s.session_task_id,
        s.sensor_data_index AS start_index,
        (
            SELECT e.sensor_data_index
            FROM preciseMarker e
            WHERE e.marker_type = 'end'
              AND e.sensor_data_index > s.sensor_data_index
              AND e.session_task_id = s.session_task_id
            ORDER BY e.sensor_data_index ASC
            LIMIT 1
        ) AS next_end_index
    FROM preciseMarker s
    WHERE s.marker_type = 'start'
    GROUP BY s.session_task_id, s.sensor_data_index
),

-- All rows that fall inside any labeled segment (gesture samples)
gesture_samples AS (
    SELECT 
        sd.id,
        sd.session_task_id,
        sd.session_task_row_index,
        sd.timestamp,
        sd.button_pressed,
        sd.motor_angle,
        sd.touch_1_position,
        sd.touch_1_pressure,
        sd.touch_1_channel,
        sd.touch_2_position,
        sd.touch_2_pressure,
        sd.touch_2_channel,
        sd.touch_3_position,
        sd.touch_3_pressure,
        sd.touch_3_channel,
        sd.touch_4_position,
        sd.touch_4_pressure,
        sd.touch_4_channel,
        sd.touch_5_position,
        sd.touch_5_pressure,
        sd.touch_5_channel,
        rt.title           AS gesture,
        rtParent.title     AS parent_gesture,
        CASE WHEN rtParent.title = 'No Gesture' THEN 0 ELSE 1 END AS is_gesture,
        -- group_id = timestamp at start of the segment (for later reference)
        (
            SELECT sdd.timestamp
            FROM sensorData sdd
            WHERE sdd.session_task_id = sd.session_task_id
              AND sdd.session_task_row_index = sei.start_index
        ) AS group_id
    FROM sensorData sd
    JOIN start_end_indices sei
      ON sd.session_task_id = sei.session_task_id
     AND sd.session_task_row_index >= sei.start_index
     AND sd.session_task_row_index <= sei.next_end_index
    LEFT JOIN sessionTask st
      ON sd.session_task_id = st.id
    LEFT JOIN recordingTask rt
      ON st.recording_task_id = rt.id
    LEFT JOIN recordingTask rtParent
      ON rt.parent_id = rtParent.id
),

-- All sensorData rows from the same sessions, labeled as "no gesture"
-- EXCEPT those already covered by gesture_samples
no_gesture_samples AS (
    SELECT 
        sd.id,
        sd.session_task_id,
        sd.session_task_row_index,
        sd.timestamp,
        sd.button_pressed,
        sd.motor_angle,
        sd.touch_1_position,
        sd.touch_1_pressure,
        sd.touch_1_channel,
        sd.touch_2_position,
        sd.touch_2_pressure,
        sd.touch_2_channel,
        sd.touch_3_position,
        sd.touch_3_pressure,
        sd.touch_3_channel,
        sd.touch_4_position,
        sd.touch_4_pressure,
        sd.touch_4_channel,
        sd.touch_5_position,
        sd.touch_5_pressure,
        sd.touch_5_channel,
        'No Gesture'       AS gesture,
        'No Gesture'       AS parent_gesture,
        0                  AS is_gesture,
        NULL               AS group_id
    FROM sensorData sd
    -- Restrict to session_task_ids that appear in labeled data
    WHERE sd.session_task_id IN (
        SELECT DISTINCT session_task_id FROM start_end_indices
    )
      AND NOT EXISTS (
          SELECT 1
          FROM start_end_indices sei
          WHERE sd.session_task_id = sei.session_task_id
            AND sd.session_task_row_index >= sei.start_index
            AND sd.session_task_row_index <= sei.next_end_index
      )
)

-- Final combined dataset: all gesture + all no-gesture
SELECT
    id,
    session_task_id,
    session_task_row_index,
    timestamp,
    button_pressed,
    motor_angle,
    touch_1_position,
    touch_1_pressure,
    touch_1_channel,
    touch_2_position,
    touch_2_pressure,
    touch_2_channel,
    touch_3_position,
    touch_3_pressure,
    touch_3_channel,
    touch_4_position,
    touch_4_pressure,
    touch_4_channel,
    touch_5_position,
    touch_5_pressure,
    touch_5_channel,
    gesture,
    parent_gesture,
    is_gesture,
    group_id
FROM gesture_samples

UNION ALL

SELECT
    id,
    session_task_id,
    session_task_row_index,
    timestamp,
    button_pressed,
    motor_angle,
    touch_1_position,
    touch_1_pressure,
    touch_1_channel,
    touch_2_position,
    touch_2_pressure,
    touch_2_channel,
    touch_3_position,
    touch_3_pressure,
    touch_3_channel,
    touch_4_position,
    touch_4_pressure,
    touch_4_channel,
    touch_5_position,
    touch_5_pressure,
    touch_5_channel,
    gesture,
    parent_gesture,
    is_gesture,
    group_id
FROM no_gesture_samples
ORDER BY session_task_id, session_task_row_index;
"""

with sqlite3.connect(db_path) as conn:
    # Use pandas to execute the query and load results into a DataFrame
    df = pd.read_sql_query(query, conn)

df.head()

,id,session_task_id,session_task_row_index,timestamp,button_pressed,motor_angle,touch_1_position,touch_1_pressure,touch_1_channel,touch_2_position,touch_2_pressure,touch_2_channel,touch_3_position,touch_3_pressure,touch_3_channel,touch_4_position,touch_4_pressure,touch_4_channel,touch_5_position,touch_5_pressure,touch_5_channel,gesture,parent_gesture,is_gesture,group_id
0,b5e548d8-203e-47f0-b702-d1ccc2ec8d2f,00939e3d-4a5f-4dd7-9bf9-de608222ef04,0,2025-10-03 16:37:53,0,184.268295,2.871814,853.0,3.0,5.046055,357.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No Gesture,No Gesture,0,None
1,0f01ab50-7419-4e90-9303-6f714e120ee2,00939e3d-4a5f-4dd7-9bf9-de608222ef04,1,2025-10-03 16:37:53,0,184.269440,2.860524,913.0,3.0,5.075835,443.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No Gesture,No Gesture,0,None
2,19e590d8-a0cd-4421-bd73-eb44552194a0,00939e3d-4a5f-4dd7-9bf9-de608222ef04,2,2025-10-03 16:37:53,0,184.271744,2.845071,947.0,4.0,5.100462,481.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No Gesture,No Gesture,0,None
3,9c2a60fc-a50e-4272-8d98-88c542b4e6c8,00939e3d-4a5f-4dd7-9bf9-de608222ef04,3,2025-10-03 16:37:53,0,184.273285,2.832931,981.0,4.0,5.127452,494.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No Gesture,No Gesture,0,None
4,3df313c7-daeb-4b01-851c-eecd2af61299,00939e3d-4a5f-4dd7-9bf9-de608222ef04,4,2025-10-03 16:37:53,0,184.276733,2.822655,1019.0,4.0,5.150771,545.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No Gesture,No Gesture,0,None


In [4]:
print(f"Data loaded: {df.shape[0]} rows, {df.shape[1]} columns")

Data loaded: 234823 rows, 25 columns


In [5]:
df['timestamp'] = pd.to_datetime(df['timestamp'])
df['time_diff'] = df.groupby(['session_task_id', 'timestamp']).cumcount()
df['recordings_per_second'] = df.groupby(['session_task_id', 'timestamp'])['id'].transform('count')
df['estimated_timestamp'] = df['timestamp'] + pd.to_timedelta(df['time_diff'] / df['recordings_per_second'], unit='s')
df = df.drop(columns=['time_diff', 'recordings_per_second'])

df["timestamp_ms"] = (df["estimated_timestamp"].astype(np.int64) // 10**6).astype(np.int64)

In [6]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 234823 entries, 0 to 234822
Data columns (total 27 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   id                      234823 non-null  object        
 1   session_task_id         234823 non-null  object        
 2   session_task_row_index  234823 non-null  int64         
 3   timestamp               234823 non-null  datetime64[ns]
 4   button_pressed          234823 non-null  int64         
 5   motor_angle             234823 non-null  float64       
 6   touch_1_position        155016 non-null  float64       
 7   touch_1_pressure        155016 non-null  float64       
 8   touch_1_channel         155016 non-null  float64       
 9   touch_2_position        128011 non-null  float64       
 10  touch_2_pressure        128011 non-null  float64       
 11  touch_2_channel         128011 non-null  float64       
 12  touch_3_position        65971 

In [7]:
df.describe()

,session_task_row_index,timestamp,button_pressed,motor_angle,touch_1_position,touch_1_pressure,touch_1_channel,touch_2_position,touch_2_pressure,touch_2_channel,touch_3_position,touch_3_pressure,touch_3_channel,touch_4_position,touch_4_pressure,touch_4_channel,touch_5_position,touch_5_pressure,touch_5_channel,is_gesture,estimated_timestamp,timestamp_ms
count,234823.000000,234823,234823.000000,234823.000000,155016.000000,155016.000000,155016.000000,128011.000000,128011.000000,128011.000000,65971.000000,65971.000000,65971.000000,31114.000000,31114.000000,31114.000000,1224.000000,1224.000000,1224.000000,234823.000000,234823,2.348230e+05
mean,1341.176252,2025-10-05 12:35:21.369265152,0.004075,117.734568,1.964881,1246.815058,3.752613,3.296028,1249.865699,3.790245,3.808662,1274.856664,3.779327,4.321962,1274.599537,3.831266,4.718806,865.544118,3.397059,0.114435,2025-10-05 12:35:21.862454016,1.759668e+12
min,0.000000,2025-10-03 15:34:58,0.000000,-3.979530,0.001231,41.000000,1.000000,0.000377,41.000000,1.000000,0.000000,41.000000,1.000000,0.000045,41.000000,1.000000,0.001951,56.000000,1.000000,0.000000,2025-10-03 15:34:58,1.759506e+12
25%,564.000000,2025-10-03 16:20:54,0.000000,82.489052,0.812911,843.000000,3.000000,2.538019,811.000000,3.000000,3.506781,858.000000,3.000000,4.085712,931.000000,3.000000,5.010641,532.500000,3.000000,0.000000,2025-10-03 16:20:54.743055616,1.759508e+12
50%,1129.000000,2025-10-03 16:41:52,0.000000,114.908966,1.829131,1148.000000,4.000000,3.300739,1156.000000,4.000000,3.947433,1166.000000,4.000000,4.970456,1127.000000,4.000000,5.699372,798.000000,3.000000,0.000000,2025-10-03 16:41:52.320000,1.759510e+12
75%,1787.000000,2025-10-07 13:45:49,0.000000,174.901047,2.880650,1599.000000,4.000000,4.238303,1580.000000,4.000000,5.102700,1634.000000,4.000000,5.939843,1514.000000,4.000000,5.996174,1088.250000,4.000000,0.000000,2025-10-07 13:45:49.798610944,1.759845e+12
max,7535.000000,2025-10-07 14:05:06,1.000000,193.055328,6.282824,3711.000000,10.000000,6.282874,4864.000000,11.000000,6.282779,3423.000000,7.000000,6.283051,3177.000000,8.000000,6.282785,2464.000000,7.000000,1.000000,2025-10-07 14:05:06.900000,1.759846e+12
std,1091.799180,NaN,0.063709,61.614215,1.318566,629.090301,1.055973,1.607579,654.482298,1.033729,1.645665,626.791021,0.938825,2.155444,590.615896,0.896253,2.134171,471.487524,0.968443,0.318340,NaN,1.680993e+08


In [8]:
# Check the distribution of is_gesture
print("\nCLASS DISTRIBUTION:")
class_counts = df['is_gesture'].value_counts().sort_index()
print(class_counts)

total_samples = len(df)
print(f"\nTotal samples: {total_samples:,}")

# Calculate percentages
for label in [0, 1]:
    count = class_counts.get(label, 0)
    percentage = (count / total_samples) * 100
    class_name = "No Gesture" if label == 0 else "Gesture"
    print(f"  {class_name} (is_gesture={label}): {count:,} samples ({percentage:.2f}%)")

# Calculate imbalance ratio
gesture_count = class_counts.get(1, 0)
no_gesture_count = class_counts.get(0, 0)

if gesture_count > 0:
    imbalance_ratio = no_gesture_count / gesture_count
    print(f"\n Ratio: {imbalance_ratio:.2f}:1")
    print(f"   (No-Gesture : Gesture)")


CLASS DISTRIBUTION:
is_gesture
0    207951
1     26872
Name: count, dtype: int64

Total samples: 234,823
  No Gesture (is_gesture=0): 207,951 samples (88.56%)
  Gesture (is_gesture=1): 26,872 samples (11.44%)

 Ratio: 7.74:1
   (No-Gesture : Gesture)


In [9]:
print("\nGesture distribution (for gestures):")
parent_breakdown = df[df['is_gesture'] == 1].groupby('parent_gesture').size().sort_values(ascending=False)
for parent_name, count in parent_breakdown.items():
    percentage = (count / gesture_count) * 100
    print(f"  {parent_name}: {count:,} samples ({percentage:.1f}%)")



Gesture distribution (for gestures):
  Tap and Hold: 6,326 samples (23.5%)
  Pinch Swipe: 5,690 samples (21.2%)
  Spread Swipe: 4,866 samples (18.1%)
  Single Swipe: 3,217 samples (12.0%)
  Two-Finger Double Tap: 1,571 samples (5.8%)
  Two-Finger Opposite Double Tap: 1,479 samples (5.5%)
  Double Tap: 1,373 samples (5.1%)
  Two-Finger Opposite Tap: 842 samples (3.1%)
  Two-Finger Tap: 785 samples (2.9%)
  Single Tap: 723 samples (2.7%)


In [10]:
SAMPLING_RATE_HZ = 60
WINDOW_SIZE_MS = 150
WINDOW_OVERLAP_PERCENT = 60

WINDOW_SIZE_SAMPLES = int((WINDOW_SIZE_MS / 1000) * SAMPLING_RATE_HZ)
STEP_SIZE_SAMPLES = int(WINDOW_SIZE_SAMPLES * (1 - WINDOW_OVERLAP_PERCENT / 100))

print(f"\nWINDOW PARAMETERS:")
print(f"  • Sampling rate: {SAMPLING_RATE_HZ} Hz")
print(f"  • Window size: {WINDOW_SIZE_MS} ms ({WINDOW_SIZE_SAMPLES} samples)")
print(f"  • Window overlap: {WINDOW_OVERLAP_PERCENT}%")
print(f"  • Step size: {STEP_SIZE_SAMPLES} samples")
print(f"  • Updates per second: {1000 / (STEP_SIZE_MS := STEP_SIZE_SAMPLES/SAMPLING_RATE_HZ*1000):.1f}")


WINDOW PARAMETERS:
  • Sampling rate: 60 Hz
  • Window size: 150 ms (9 samples)
  • Window overlap: 60%
  • Step size: 3 samples
  • Updates per second: 20.0


In [11]:
# Same sensor columns as main pipeline
SENSOR_COLUMNS = [
    'button_pressed',
    'motor_angle',
    'touch_1_position', 'touch_1_pressure', 'touch_1_channel',
    'touch_2_position', 'touch_2_pressure', 'touch_2_channel',
    'touch_3_position', 'touch_3_pressure', 'touch_3_channel',
    'touch_4_position', 'touch_4_pressure', 'touch_4_channel',
    'touch_5_position', 'touch_5_pressure', 'touch_5_channel',
]


In [12]:
df_binary = df.copy()
df_binary['timestamp'] = pd.to_datetime(df_binary['timestamp'])
df_binary = df_binary.sort_values(['session_task_id', 'timestamp']).reset_index(drop=True)

# Handle missing values
for col in SENSOR_COLUMNS:
    if col in df_binary.columns:
        df_binary[col] = df_binary.groupby('session_task_id')[col].fillna(method='ffill')
        df_binary[col] = df_binary.groupby('session_task_id')[col].fillna(method='bfill')

df_binary[SENSOR_COLUMNS] = df_binary[SENSOR_COLUMNS].fillna(0)
df_binary['button_pressed'] = df_binary['button_pressed'].astype(int)

In [13]:
def create_sliding_windows_binary(session_df, window_size, step_size):
    session_df = session_df.sort_values('timestamp').reset_index(drop=True)
    sensor_data = session_df[SENSOR_COLUMNS].values
    binary_labels = session_df['is_gesture'].values
    
    n_samples = len(sensor_data)
    windows = []
    window_labels = []
    window_metadata = []
    
    for start_idx in range(0, n_samples - window_size + 1, step_size):
        end_idx = start_idx + window_size
        
        window = sensor_data[start_idx:end_idx]
        window_label_segment = binary_labels[start_idx:end_idx]
        
        # MAJORITY VOTING: If >50% of window contains gesture, label as gesture
        # For binary detector, we want high recall, so even small gesture presence counts
        gesture_percentage = np.mean(window_label_segment)
        
        # Threshold: if >30% of window is gesture, label as gesture
        # This makes detector more sensitive (fewer false negatives)
        window_label = 1 if gesture_percentage > 0.3 else 0
        
        windows.append(window)
        window_labels.append(window_label)
        window_metadata.append({
            'session_task_id': session_df['session_task_id'].iloc[0],
            'start_idx': start_idx,
            'end_idx': end_idx,
            'gesture_percentage': gesture_percentage
        })
    
    return np.array(windows), np.array(window_labels), window_metadata


# Apply to all sessions
all_windows = []
all_labels = []
all_metadata = []

session_groups = df_binary.groupby('session_task_id')
total_sessions = len(session_groups)

print(f"Processing {total_sessions} sessions...")

for idx, (session_id, session_df) in enumerate(session_groups):
    if (idx + 1) % 10 == 0:
        print(f"  Processing session {idx + 1}/{total_sessions}...")
    
    if len(session_df) >= WINDOW_SIZE_SAMPLES:  # Only process if enough samples
        windows, labels, metadata = create_sliding_windows_binary(
            session_df,
            WINDOW_SIZE_SAMPLES,
            STEP_SIZE_SAMPLES
        )
        
        all_windows.extend(windows)
        all_labels.extend(labels)
        all_metadata.extend(metadata)

# Convert to numpy arrays
X_windows = np.array(all_windows)
y_binary = np.array(all_labels)

print(f"\nWindowing complete!")
print(f"  Total windows: {len(X_windows):,}")
print(f"  Window shape: {X_windows.shape}")
print(f"  (n_windows, window_size, n_features)")

# Check final class distribution
unique, counts = np.unique(y_binary, return_counts=True)
print(f"\nWINDOW LABEL DISTRIBUTION:")
for label, count in zip(unique, counts):
    percentage = (count / len(y_binary)) * 100
    label_name = "No Gesture" if label == 0 else "Gesture"
    print(f"  {label_name}: {count:,} windows ({percentage:.1f}%)")

window_ratio = counts[0] / counts[1] if len(counts) > 1 else 0
print(f"  Window ratio: {window_ratio:.2f}:1")


Processing 104 sessions...
  Processing session 10/104...
  Processing session 20/104...
  Processing session 30/104...
  Processing session 40/104...
  Processing session 50/104...
  Processing session 60/104...
  Processing session 70/104...
  Processing session 80/104...
  Processing session 90/104...
  Processing session 100/104...

Windowing complete!
  Total windows: 78,033
  Window shape: (78033, 9, 17)
  (n_windows, window_size, n_features)

WINDOW LABEL DISTRIBUTION:
  No Gesture: 67,458 windows (86.4%)
  Gesture: 10,575 windows (13.6%)
  Window ratio: 6.38:1


In [14]:
def extract_simple_features(window):
    features = []
    
    # Assume sensor column order from SENSOR_COLUMNS
    # Indices: button=0, motor=1, touch_pos=[2,5,8,11,14], touch_press=[3,6,9,12,15]
    button_col = 0
    motor_col = 1
    touch_pressure_cols = [3, 6, 9, 12, 15]  # touch_X_pressure indices
    touch_position_cols = [2, 5, 8, 11, 14]  # touch_X_position indices
    
    # === BUTTON FEATURES (2 features) ===
    features.append(np.any(window[:, button_col]))  # Button pressed at all?
    features.append(np.mean(window[:, button_col]))  # Button press percentage
    
    # === MOTOR ANGLE FEATURES (4 features) ===
    motor_data = window[:, motor_col]
    features.append(np.max(motor_data))              # Max angle
    features.append(np.std(motor_data))              # Motion variance
    features.append(np.ptp(motor_data))              # Range (peak-to-peak)
    
    # Motion magnitude (simple velocity approximation)
    if len(motor_data) > 1:
        motion_diff = np.abs(np.diff(motor_data))
        features.append(np.sum(motion_diff))         # Total motion
    else:
        features.append(0)
    
    # === TOUCH ACTIVITY FEATURES (7 features) ===
    # Count active touches per timestep
    touch_pressures = window[:, touch_pressure_cols]
    active_touches = (touch_pressures > 0).astype(int)
    
    features.append(np.max(np.sum(active_touches, axis=1)))  # Max simultaneous touches
    features.append(np.mean(np.sum(active_touches, axis=1))) # Avg active touches
    features.append(np.any(active_touches))                  # Any touch at all?
    
    # Total pressure across all sensors
    features.append(np.sum(touch_pressures))                 # Total pressure
    features.append(np.max(touch_pressures))                 # Max pressure
    
    # Touch movement (position change)
    touch_positions = window[:, touch_position_cols]
    if len(touch_positions) > 1:
        position_diff = np.abs(np.diff(touch_positions, axis=0))
        features.append(np.sum(position_diff))               # Total touch movement
        features.append(np.max(position_diff))               # Max position jump
    else:
        features.append(0)
        features.append(0)
    
    return np.array(features, dtype=np.float32)


# Extract features for all windows
print("Extracting simple features from windows...")

import time
start_time = time.time()

X_features_simple = np.array([extract_simple_features(window) for window in X_windows])

extraction_time = time.time() - start_time
time_per_window = (extraction_time / len(X_windows)) * 1000  # ms

print(f"\n Feature extraction complete!")
print(f"  Feature matrix shape: {X_features_simple.shape}")
print(f"  Features per window: {X_features_simple.shape[1]}")
print(f"  Total extraction time: {extraction_time:.2f}s")
print(f"  Time per window: {time_per_window:.4f}ms")

print(f"\n FEATURE BREAKDOWN:")
print(f"  Button features: 2")
print(f"  Motor angle features: 4")
print(f"  Touch activity features: 7")
print(f"  TOTAL: {X_features_simple.shape[1]} simple features")


Extracting simple features from windows...

 Feature extraction complete!
  Feature matrix shape: (78033, 13)
  Features per window: 13
  Total extraction time: 3.90s
  Time per window: 0.0500ms

 FEATURE BREAKDOWN:
  Button features: 2
  Motor angle features: 4
  Touch activity features: 7
  TOTAL: 13 simple features


In [15]:
from sklearn.model_selection import train_test_split

# Create metadata DataFrame
metadata_df = pd.DataFrame(all_metadata)

# Get unique sessions
unique_sessions = metadata_df['session_task_id'].unique()
print(f"Total sessions: {len(unique_sessions)}")

# Split by session (not by window!)
train_sessions, test_sessions = train_test_split(
    unique_sessions,
    test_size=0.2,
    random_state=42
)

print(f"\n Split sessions:")
print(f"  Training sessions: {len(train_sessions)}")
print(f"  Test sessions: {len(test_sessions)}")

# Get window indices
train_mask = metadata_df['session_task_id'].isin(train_sessions)
test_mask = metadata_df['session_task_id'].isin(test_sessions)

# Split data
X_train = X_features_simple[train_mask]
X_test = X_features_simple[test_mask]
y_train = y_binary[train_mask]
y_test = y_binary[test_mask]

print(f"\n Windows split:")
print(f"  Training windows: {len(X_train):,} ({len(X_train)/len(X_features_simple)*100:.1f}%)")
print(f"  Test windows: {len(X_test):,} ({len(X_test)/len(X_features_simple)*100:.1f}%)")

print(f"\n TRAINING SET:")
unique_train, counts_train = np.unique(y_train, return_counts=True)
for label, count in zip(unique_train, counts_train):
    label_name = "No Gesture" if label == 0 else "Gesture"
    print(f"  {label_name}: {count:,} ({count/len(y_train)*100:.1f}%)")

print(f"\n TEST SET:")
unique_test, counts_test = np.unique(y_test, return_counts=True)
for label, count in zip(unique_test, counts_test):
    label_name = "No Gesture" if label == 0 else "Gesture"
    print(f"  {label_name}: {count:,} ({count/len(y_test)*100:.1f}%)")


Total sessions: 104

 Split sessions:
  Training sessions: 83
  Test sessions: 21

 Windows split:
  Training windows: 62,184 (79.7%)
  Test windows: 15,849 (20.3%)

 TRAINING SET:
  No Gesture: 54,416 (87.5%)
  Gesture: 7,768 (12.5%)

 TEST SET:
  No Gesture: 13,042 (82.3%)
  Gesture: 2,807 (17.7%)


In [16]:
from sklearn.preprocessing import StandardScaler

# Scale features (fit on train only)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"Features scaled using StandardScaler")
print(f"  Training mean: {X_train_scaled.mean():.4f}")
print(f"  Training std: {X_train_scaled.std():.4f}")


Features scaled using StandardScaler
  Training mean: 0.0000
  Training std: 1.0000


In [17]:
from sklearn.utils.class_weight import compute_class_weight

# Compute class weights
class_weights = compute_class_weight(
    'balanced',
    classes=np.array([0, 1]),
    y=y_train
)

class_weight_dict = {0: class_weights[0], 1: class_weights[1]}

print(f"Calculated class weights:")
print(f"  No-Gesture (0): {class_weight_dict[0]:.3f}")
print(f"  Gesture (1): {class_weight_dict[1]:.3f}")
print(f"  Ratio: {class_weight_dict[1]/class_weight_dict[0]:.2f}x higher weight for gestures")

# For XGBoost
scale_pos_weight = class_weight_dict[1] / class_weight_dict[0]
print(f"\n  XGBoost scale_pos_weight: {scale_pos_weight:.3f}")


Calculated class weights:
  No-Gesture (0): 0.571
  Gesture (1): 4.003
  Ratio: 7.01x higher weight for gestures

  XGBoost scale_pos_weight: 7.005


In [18]:
# Import classifiers
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier

# Evaluation metrics and threshold optimization
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, classification_report, roc_curve, auc,
    precision_recall_curve, roc_auc_score
)
from sklearn.model_selection import cross_val_score
import time


In [19]:
# Dictionary to store all model results
binary_results = {}

def evaluate_binary_model(model, X_test, y_test, model_name, X_train=None, y_train=None):
    # Make predictions
    y_pred = model.predict(X_test)
    
    # Get prediction probabilities for threshold optimization
    if hasattr(model, 'predict_proba'):
        y_proba = model.predict_proba(X_test)[:, 1]  # Probability of gesture class
    else:
        y_proba = model.decision_function(X_test)  # For SVM
        # Normalize to [0, 1]
        y_proba = (y_proba - y_proba.min()) / (y_proba.max() - y_proba.min())
    
    # Calculate metrics at default threshold (0.5)
    test_accuracy = accuracy_score(y_test, y_pred)
    test_precision = precision_score(y_test, y_pred, zero_division=0)
    test_recall = recall_score(y_test, y_pred, zero_division=0)
    test_f1 = f1_score(y_test, y_pred, zero_division=0)
    test_roc_auc = roc_auc_score(y_test, y_proba)
    
    # Training accuracy if provided
    train_accuracy = None
    if X_train is not None and y_train is not None:
        y_train_pred = model.predict(X_train)
        train_accuracy = accuracy_score(y_train, y_train_pred)
    
    # Confusion matrix
    cm = confusion_matrix(y_test, y_pred)
    
    # Calculate confusion matrix metrics
    tn, fp, fn, tp = cm.ravel()
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0  # True negative rate
    
    # Store results
    binary_results[model_name] = {
        'model': model,
        'y_pred': y_pred,
        'y_proba': y_proba,
        'accuracy': test_accuracy,
        'precision': test_precision,
        'recall': test_recall,
        'f1': test_f1,
        'roc_auc': test_roc_auc,
        'specificity': specificity,
        'train_accuracy': train_accuracy,
        'confusion_matrix': cm,
        'classification_report': classification_report(y_test, y_pred),
        'tn': tn, 'fp': fp, 'fn': fn, 'tp': tp
    }
    
    return binary_results[model_name]


def print_binary_results(model_name, result):
    print(f"\n{'=' * 70}")
    print(f"MODEL: {model_name}")
    print(f"{'=' * 70}")
    
    if result['train_accuracy'] is not None:
        print(f"Training Accuracy:    {result['train_accuracy']:.4f}")
    
    print(f"Test Accuracy:        {result['accuracy']:.4f}")
    print(f"Recall (Sensitivity): {result['recall']:.4f}  ← Key metric!")
    print(f"Precision:            {result['precision']:.4f}")
    print(f"Specificity:          {result['specificity']:.4f}")
    print(f"F1-Score:             {result['f1']:.4f}")
    print(f"ROC-AUC:              {result['roc_auc']:.4f}")
    
    print(f"\n🎯 CONFUSION MATRIX:")
    cm = result['confusion_matrix']
    print(f"  True Negatives (TN):  {result['tn']:,}  (correctly rejected no-gesture)")
    print(f"  False Positives (FP): {result['fp']:,}  (false alarms)")
    print(f"  False Negatives (FN): {result['fn']:,}  (MISSED GESTURES - want to minimize!)")
    print(f"  True Positives (TP):  {result['tp']:,}  (correctly detected gestures)")
    
    print(f"\n{result['classification_report']}")


In [20]:
print("LIGHTWEIGHT RANDOM FOREST")

# Train lightweight RF (fewer trees = faster)
rf_params = {
    'n_estimators': 15,        # Very lightweight
    'max_depth': 5,             # Shallow trees
    'min_samples_split': 10,
    'min_samples_leaf': 5,
    'max_features': 'sqrt',
    'random_state': 42,
    'n_jobs': -1,
    'class_weight': 'balanced'  # Handle imbalance
}

print(f"\nTraining parameters: {rf_params}")

start_time = time.time()
rf_detector = RandomForestClassifier(**rf_params)
rf_detector.fit(X_train_scaled, y_train)
rf_train_time = time.time() - start_time

print(f"Training complete in {rf_train_time:.2f}s")

# Evaluate
print("\nEvaluating on test set...")
start_time = time.time()
rf_result = evaluate_binary_model(rf_detector, X_test_scaled, y_test, 
                                   'Random Forest', X_train_scaled, y_train)
rf_inference_time = (time.time() - start_time) / len(X_test_scaled) * 1000  # ms per sample

print_binary_results('Random Forest', rf_result)
print(f"\nInference time: {rf_inference_time:.4f} ms per window")


LIGHTWEIGHT RANDOM FOREST

Training parameters: {'n_estimators': 15, 'max_depth': 5, 'min_samples_split': 10, 'min_samples_leaf': 5, 'max_features': 'sqrt', 'random_state': 42, 'n_jobs': -1, 'class_weight': 'balanced'}
Training complete in 0.09s

Evaluating on test set...

MODEL: Random Forest
Training Accuracy:    0.7856
Test Accuracy:        0.7992
Recall (Sensitivity): 0.8821  ← Key metric!
Precision:            0.4648
Specificity:          0.7814
F1-Score:             0.6088
ROC-AUC:              0.9005

🎯 CONFUSION MATRIX:
  True Negatives (TN):  10,191  (correctly rejected no-gesture)
  False Positives (FP): 2,851  (false alarms)
  False Negatives (FN): 331  (MISSED GESTURES - want to minimize!)
  True Positives (TP):  2,476  (correctly detected gestures)

              precision    recall  f1-score   support

           0       0.97      0.78      0.86     13042
           1       0.46      0.88      0.61      2807

    accuracy                           0.80     15849
   macro 

In [21]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Reshape features for CNN (add channel dimension)
X_train_cnn = X_train_scaled.reshape(-1, X_train_scaled.shape[1], 1)
X_test_cnn = X_test_scaled.reshape(-1, X_test_scaled.shape[1], 1)

# Build tiny model
cnn_detector = keras.Sequential([
    layers.Conv1D(8, 3, activation='relu', input_shape=(X_train_cnn.shape[1], 1)),
    layers.GlobalAveragePooling1D(),
    layers.Dense(4, activation='relu'),
    layers.Dense(1, activation='sigmoid')  # Binary output
])

cnn_detector.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy', keras.metrics.Recall(name='recall')]
)

print(cnn_detector.summary())

# Train with class weights
class_weight_dict = {0: class_weight_dict[0], 1: class_weight_dict[1]}

print(f"\nTraining CNN with class weights: {class_weight_dict}")

history = cnn_detector.fit(
    X_train_cnn, y_train,
    epochs=20,
    batch_size=32,
    validation_split=0.2,
    class_weight=class_weight_dict,
    verbose=0
)

# Get predictions as probabilities
y_proba_cnn = cnn_detector.predict(X_test_cnn, verbose=0).flatten()
y_pred_cnn = (y_proba_cnn > 0.5).astype(int)

# Manual evaluation
from sklearn.metrics import accuracy_score, recall_score, precision_score, confusion_matrix

cm_cnn = confusion_matrix(y_test, y_pred_cnn)
tn, fp, fn, tp = cm_cnn.ravel()

binary_results['Tiny CNN'] = {
    'model': cnn_detector,
    'y_pred': y_pred_cnn,
    'y_proba': y_proba_cnn,
    'accuracy': accuracy_score(y_test, y_pred_cnn),
    'recall': recall_score(y_test, y_pred_cnn),
    'precision': precision_score(y_test, y_pred_cnn),
    'roc_auc': roc_auc_score(y_test, y_proba_cnn),
    'confusion_matrix': cm_cnn,
    'tn': tn, 'fp': fp, 'fn': fn, 'tp': tp
}

print(f"\nCNN training complete")
print(f"  Recall: {binary_results['Tiny CNN']['recall']:.4f}")
print(f"  Precision: {binary_results['Tiny CNN']['precision']:.4f}")
print(f"  Accuracy: {binary_results['Tiny CNN']['accuracy']:.4f}")


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 11, 8)          │            32 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d        │ (None, 8)              │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 4)              │            36 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │             5 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 73 (292.00 B)

 Trainable params: 73 (292.00 B)

 Non-trainable params: 0 (0.00 B)

None

Training CNN with class weights: {0: np.float64(0.5713760658629815), 1: np.float64(4.002574665293512)}

CNN training complete
  Recall: 0.7193
  Precision: 0.4136
  Accuracy: 0.7696


In [22]:
print("\nBaseline: Any touch pressure > threshold = gesture detected")

# Simple heuristic: if sum of touch pressures > threshold
def simple_threshold_predict(X, threshold=0.5):
    predictions = []
    for sample in X:
        # Features: motor_max, motor_std, motor_range, motor_sum, etc.
        # Let's use feature 1 (motor_std) and total touch pressure
        motor_std = sample[1]  # Assuming index 1 is motor std
        touch_activity = np.sum(sample[10:])  # Last features are touch-related
        
        # Gesture if motor moving OR touches active
        is_gesture = (motor_std > 0.1) or (touch_activity > threshold)
        predictions.append(1 if is_gesture else 0)
    
    return np.array(predictions)

y_pred_baseline = simple_threshold_predict(X_test_scaled)

cm_baseline = confusion_matrix(y_test, y_pred_baseline)
tn, fp, fn, tp = cm_baseline.ravel()

binary_results['Threshold Baseline'] = {
    'model': None,
    'y_pred': y_pred_baseline,
    'y_proba': None,
    'accuracy': accuracy_score(y_test, y_pred_baseline),
    'recall': recall_score(y_test, y_pred_baseline),
    'precision': precision_score(y_test, y_pred_baseline),
    'roc_auc': None,
    'confusion_matrix': cm_baseline,
    'tn': tn, 'fp': fp, 'fn': fn, 'tp': tp
}

print(f"\nBaseline evaluation complete")
print(f"  Recall: {binary_results['Threshold Baseline']['recall']:.4f}")
print(f"  Precision: {binary_results['Threshold Baseline']['precision']:.4f}")
print(f"  Accuracy: {binary_results['Threshold Baseline']['accuracy']:.4f}")



Baseline: Any touch pressure > threshold = gesture detected

Baseline evaluation complete
  Recall: 0.3124
  Precision: 0.3967
  Accuracy: 0.7941


In [23]:
# Use the Random Forest model
model = rf_detector
y_proba = rf_result['y_proba']

# Calculate precision-recall curve
precision, recall, thresholds = precision_recall_curve(y_test, y_proba)

print(f"\n📊 Analyzing {len(thresholds)} threshold values...")

# Find threshold for different recall targets
recall_targets = [0.95, 0.98, 0.99, 0.995]

optimal_thresholds = {}

for target_recall in recall_targets:
    # Find threshold closest to target recall
    idx = np.argmin(np.abs(recall - target_recall))
    threshold = thresholds[idx] if idx < len(thresholds) else thresholds[-1]
    precision_at_threshold = precision[idx]
    
    optimal_thresholds[target_recall] = {
        'threshold': threshold,
        'precision': precision_at_threshold,
        'recall': recall[idx]
    }
    
    print(f"\nRecall ≥ {target_recall*100:.1f}%:")
    print(f"  Threshold: {threshold:.4f}")
    print(f"  Achieved recall: {recall[idx]:.4f}")
    print(f"  Precision: {precision_at_threshold:.4f}")
    print(f"  False positive rate: {1-precision_at_threshold:.1%}")

# Select target recall of 98%
TARGET_RECALL = 0.98
optimal_threshold = optimal_thresholds[TARGET_RECALL]['threshold']

print(f"\n{'='*70}")
print(f"SELECTED THRESHOLD: {optimal_threshold:.4f}")
print(f"   Target: Recall ≥ 98%")
print(f"   Achieved: Recall {optimal_thresholds[TARGET_RECALL]['recall']:.4f}")
print(f"   Precision: {optimal_thresholds[TARGET_RECALL]['precision']:.4f}")
print(f"   → Accept {(1-optimal_thresholds[TARGET_RECALL]['precision'])*100:.1f}% false positives")
print(f"{'='*70}")

# Apply optimized threshold
y_pred_optimized = (y_proba >= optimal_threshold).astype(int)

# Recalculate metrics with optimized threshold
cm_optimized = confusion_matrix(y_test, y_pred_optimized)
tn, fp, fn, tp = cm_optimized.ravel()

accuracy_optimized = accuracy_score(y_test, y_pred_optimized)
precision_optimized = precision_score(y_test, y_pred_optimized)
recall_optimized = recall_score(y_test, y_pred_optimized)
f1_optimized = f1_score(y_test, y_pred_optimized)
specificity_optimized = tn / (tn + fp) if (tn + fp) > 0 else 0

print(f"\nOPTIMIZED DETECTOR PERFORMANCE:")
print(f"  Accuracy:    {accuracy_optimized:.4f}")
print(f"  Recall:      {recall_optimized:.4f}")
print(f"  Precision:   {precision_optimized:.4f}")
print(f"  Specificity: {specificity_optimized:.4f}")
print(f"  F1-Score:    {f1_optimized:.4f}")

print(f"\nCONFUSION MATRIX:")
print(f"  True Negatives:  {tn:,}  (correct rejections)")
print(f"  False Positives: {fp:,}  (false alarms)")
print(f"  False Negatives: {fn:,}  (MISSED GESTURES)")
print(f"  True Positives:  {tp:,}  (detected gestures)")

# Store optimized threshold
binary_results['Random Forest (Optimized)'] = {
    'model': rf_detector,
    'y_pred': y_pred_optimized,
    'y_proba': y_proba,
    'threshold': optimal_threshold,
    'accuracy': accuracy_optimized,
    'precision': precision_optimized,
    'recall': recall_optimized,
    'f1': f1_optimized,
    'specificity': specificity_optimized,
    'confusion_matrix': cm_optimized,
    'tn': tn, 'fp': fp, 'fn': fn, 'tp': tp
}



📊 Analyzing 2630 threshold values...

Recall ≥ 95.0%:
  Threshold: 0.3907
  Achieved recall: 0.9501
  Precision: 0.4283
  False positive rate: 57.2%

Recall ≥ 98.0%:
  Threshold: 0.2808
  Achieved recall: 0.9800
  Precision: 0.3865
  False positive rate: 61.4%

Recall ≥ 99.0%:
  Threshold: 0.0946
  Achieved recall: 0.9900
  Precision: 0.3371
  False positive rate: 66.3%

Recall ≥ 99.5%:
  Threshold: 0.0338
  Achieved recall: 0.9950
  Precision: 0.2384
  False positive rate: 76.2%

SELECTED THRESHOLD: 0.2808
   Target: Recall ≥ 98%
   Achieved: Recall 0.9800
   Precision: 0.3865
   → Accept 61.4% false positives

OPTIMIZED DETECTOR PERFORMANCE:
  Accuracy:    0.7209
  Recall:      0.9800
  Precision:   0.3865
  Specificity: 0.6652
  F1-Score:    0.5544

CONFUSION MATRIX:
  True Negatives:  8,675  (correct rejections)
  False Positives: 4,367  (false alarms)
  False Negatives: 56  (MISSED GESTURES)
  True Positives:  2,751  (detected gestures)
